In [ ]:
pip install python-bingx matplotlib mplfinance pandas

In [ ]:
import os
from bingX import BingX
import mplfinance as mpl
import pandas as pd
from datetime import datetime


api_key = os.environ.get("BINGX_API_KEY","")
secret_key = os.environ.get("BINGX_SECRET_KEY","")


bingx_client = BingX(api_key, secret_key)



date = datetime(2024,3,31,12,0,0)
date_end = datetime(2024,4,2,12,0,0)

fecha = int(date.timestamp() * 1000)
fecha_end = int(date_end.timestamp() * 1000)

print(fecha)
intervals = ["1M","1d","4h","1h","30m","15m","5m"]
data =[bingx_client.perpetual_v2.market.get_k_line_data(symbol="BTC-USDT",interval=interval,start_time=fecha,end_time=fecha_end) for interval in intervals ]

df = pd.DataFrame(data[2])
df['time'] = pd.to_datetime(df['time'],unit='ms')
df = df.astype({'open':'float'})
df = df.astype({'close':'float'})
df = df.astype({'high':'float'})
df = df.astype({'low':'float'})
df= df.astype({'volume':'float'})

df = df.set_index('time',inplace=False)

plot, ax = mpl.plot(df,type='candle',returnfig=True)
ax[0].set_title("BTC-USDT - 4h")